# Healthcare Payer Member Churn - Deployment & Monitoring

**Part 3 of 3**: This notebook covers model deployment and production operations.

## Prerequisites
- Complete **Part 1: Feature Engineering** notebook
- Complete **Part 2: Model Training & Comparison** notebook
- Ensure you have a trained model registered as:
  - `demo.hls.mx_churn@Champion`

## What's Covered
- Batch inference
- Real-time model serving
- SQL-based inference
- Model monitoring strategies
- Databricks SQL dashboards
- Business actions and interventions
- Production roadmap

In [ ]:
---
# 8. Model Deployment Options

Databricks provides multiple deployment options for ML models:

1. **Batch Inference**: Score large datasets using Spark (covered below)
2. **Model Serving**: Real-time REST API endpoints
3. **Streaming**: Process data streams with Structured Streaming
4. **Embedded**: Export models for edge deployment

## 8.1 Batch Inference with Feature Engineering

Batch scoring is ideal for:
- Periodic member risk assessments
- Bulk predictions on enrollment data
- Offline model evaluation

# Batch inference using Feature Engineering Client
from databricks.feature_engineering import FeatureEngineeringClient
from pyspark.sql.types import DoubleType

# Load members to score (in production, this would be your active member base)
inference_df = spark.read.table("demo.hls.label_features_versioned").drop("disenrolled")

print(f"Scoring {inference_df.count():,} members...")

fe = FeatureEngineeringClient()

# Model URI with Champion alias
model_uri = f"models:/{model_name}@Champion"

# Batch score with feature engineering
# This automatically joins features from the feature store
predictions_df = fe.score_batch(
    df=inference_df,
    model_uri=model_uri,
    result_type=DoubleType()  # Returns probability scores
)

print(f"\n✓ Batch scoring complete")
print(f"\nPreview of predictions:")
display(predictions_df.limit(100))

# Save predictions for downstream use
predictions_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("demo.hls.member_churn_scores")

print("\n✓ Predictions saved to demo.hls.member_churn_scores")

In [ ]:
---
## 8.2 Real-Time Model Serving

**Model Serving** provides production-ready REST API endpoints for real-time predictions:

### Benefits:
- **Low latency**: Sub-second response times
- **Auto-scaling**: Handles variable load
- **Monitoring**: Built-in metrics and logging
- **A/B testing**: Route traffic between model versions
- **Authentication**: Secure with tokens

### Setup Options:
1. **UI**: Navigate to Serving → Create Serving Endpoint
2. **API**: Programmatically via REST API (shown below)
3. **Terraform**: Infrastructure as code

In [ ]:
# Model Serving Setup
# Note: This is informational - actual endpoint creation is typically done via UI or API

print("📍 To create a Model Serving endpoint:\n")
print("1. Navigate to the Databricks workspace")
print("2. Click 'Serving' in the left sidebar")
print("3. Click 'Create Serving Endpoint'")
print("4. Select model: demo.hls.mx_churn@Champion")
print("5. Configure:")
print("   - Workload size: Small (for testing)")
print("   - Scale to zero: Enabled (cost optimization)")
print("6. Click 'Create'\n")

print("📍 Alternative: Query using AI_QUERY function (shown below)")

%sql
-- Query model endpoint using AI_QUERY function
-- This requires a model serving endpoint named 'mx_churn' to be created first

-- Example query:
SELECT ai_query(
  'mx_churn',  -- endpoint name
  request => named_struct(
    'num_claims_total', 15,
    'num_distinct_providers', 3,
    'num_months_with_claims', 8,
    'pct_denied_claims', 0.10,
    'total_charge_amount', 25000.0,
    'total_paid_amount', 20000.0,
    'avg_allowed_amount', 1500.0,
    'avg_paid_amount', 1300.0,
    'age', 45.0,
    'avg_days_between_claims', 30.0,
    'Inpatient', 2,
    'Outpatient', 8,
    'Pharmacy', 15,
    'Professional', 10,
    'avg_allowed_paid_ratio', 1.15
  )
) AS churn_probability;

# Example JSON payload for REST API call
example_payload = {
    "num_claims_total": 15,
    "num_distinct_providers": 3,
    "num_months_with_claims": 8,
    "pct_denied_claims": 0.10,
    "total_charge_amount": 25000.0,
    "total_paid_amount": 20000.0,
    "avg_allowed_amount": 1500.0,
    "avg_paid_amount": 1300.0,
    "age": 45.0,
    "avg_days_between_claims": 30.0,
    "Inpatient": 2,
    "Outpatient": 8,
    "Pharmacy": 15,
    "Professional": 10,
    "avg_allowed_paid_ratio": 1.15
}

import json
print("Example REST API payload:")
print(json.dumps(example_payload, indent=2))

In [ ]:
---
# 9. Model Monitoring and Operations

After deployment, continuous monitoring is essential for production ML systems.

## 9.1 Key Monitoring Areas

### Model Performance:
- **Prediction accuracy**: Track precision, recall, F1 over time
- **Business metrics**: Actual vs predicted churn rates
- **Confusion matrix**: False positives vs false negatives

### Data Quality:
- **Feature drift**: Are input distributions changing?
- **Missing values**: Track NULL rates for key features
- **Outliers**: Detect anomalous input values

### System Health:
- **Latency**: Response time percentiles (p50, p95, p99)
- **Throughput**: Requests per second
- **Error rates**: Failed predictions
- **Resource usage**: CPU, memory, costs

In [ ]:
---
## 9.2 Databricks SQL Dashboards

Create operational dashboards to monitor model performance:

### Dashboard Components:

**1. Churn Risk Distribution**
```sql
SELECT 
  CASE 
    WHEN prediction > 0.7 THEN 'High Risk'
    WHEN prediction > 0.3 THEN 'Medium Risk'
    ELSE 'Low Risk'
  END as risk_category,
  COUNT(*) as member_count
FROM demo.hls.member_churn_scores
GROUP BY risk_category
```

**2. Top Risk Members**
```sql
SELECT 
  member_id,
  prediction as churn_probability,
  num_claims_total,
  age
FROM demo.hls.member_churn_scores
ORDER BY prediction DESC
LIMIT 100
```

**3. Feature Importance Trends**
- Track which features drive predictions over time
- Identify changing patterns in member behavior

---
## 9.3 Databricks Lakehouse Monitoring

**Lakehouse Monitoring** (formerly known as Model Monitoring) provides automated drift detection:

### Setup:
```python
# Example: Create a monitor for your predictions table
import databricks.lakehouse_monitoring as lm

lm.create_monitor(
    table_name="demo.hls.member_churn_scores",
    granularities=["1 day"],  # Monitoring frequency
    output_schema_name="demo.hls_monitoring",
    baseline_table_name="demo.hls.member_churn_baseline"  # Historical baseline
)
```

### Monitors Track:
- **Statistical drift**: Distribution changes in features
- **Prediction drift**: Changes in model output distribution
- **Label drift**: If actual labels become available
- **Data quality**: Missing values, schema changes

### Alerts:
- Configure email/Slack notifications
- Set thresholds for acceptable drift
- Trigger retraining pipelines automatically

---
## 9.4 Databricks Genie (Conversational AI)

**Databricks Genie** enables natural language interaction with your churn predictions:

### Example Questions:
- "Show me members with high churn risk in California"
- "What's the average churn probability by age group?"
- "Which members have the highest predicted churn and low engagement?"
- "Compare churn rates between pharmacy and medical users"

### Business Value:
- **Democratize insights**: Non-technical users can query predictions
- **Faster decisions**: Natural language vs writing SQL
- **Exploratory analysis**: Quickly test hypotheses

### Setup:
1. Navigate to your workspace
2. Click on **Genie** in the left sidebar
3. Connect to your schema (demo.hls)
4. Start asking questions!

Genie can automatically:
- Generate appropriate SQL queries
- Create visualizations
- Provide insights and explanations

---
# 10. Business Actions and Interventions

The ultimate goal is to use predictions to **reduce member churn**. Here's how:

## 10.1 Risk-Based Interventions

### High Risk Members (probability > 0.7):
- **Personal outreach**: Dedicated care coordinator contact
- **Retention offers**: Premium waivers, enhanced benefits
- **Root cause analysis**: Why are they considering leaving?
- **Expedited issue resolution**: Fast-track any complaints

### Medium Risk Members (0.3 - 0.7):
- **Automated engagement**: Email campaigns highlighting benefits
- **Education**: Explain unused benefits and services
- **Preventive care nudges**: Wellness checkups, screenings
- **Satisfaction surveys**: Proactive feedback collection

### Low Risk Members (< 0.3):
- **Standard engagement**: Regular newsletters
- **Advocacy programs**: Turn satisfied members into advocates
- **Upsell opportunities**: Additional coverage options

## 10.2 Measuring Impact

Track the effectiveness of interventions:
- **Retention rate**: % of at-risk members who stayed
- **Cost per retention**: Intervention cost vs lifetime value
- **Time to churn**: Did interventions delay disenrollment?
- **ROI**: Value saved from reduced churn

---
# 11. Summary and Next Steps

## ✅ What We've Built

This end-to-end ML solution demonstrates:

1. **Feature Engineering**: Scalable feature pipelines with Databricks Feature Store
2. **Model Training**: Multiple algorithms with MLflow tracking
3. **Model Comparison**: Rigorous evaluation with Champion/Challenger pattern
4. **Model Registry**: Centralized governance with Unity Catalog
5. **Deployment**: Batch and real-time inference options
6. **Monitoring**: Comprehensive observability and drift detection

## 🚀 Recommended Next Steps

### Short Term (1-2 weeks):
1. **Deploy to production**: Set up model serving endpoint
2. **Schedule batch scoring**: Daily predictions for all active members
3. **Create dashboards**: Visualize churn risk trends
4. **Integrate with CRM**: Send high-risk members to retention team

### Medium Term (1-2 months):
1. **A/B test interventions**: Measure impact of retention programs
2. **Add more features**: Social determinants, claims patterns, satisfaction scores
3. **Segment models**: Build specialized models by plan type or region
4. **Explainability**: Add SHAP values to explain individual predictions

### Long Term (3-6 months):
1. **AutoML integration**: Automate model selection and hyperparameter tuning
2. **Real-time features**: Streaming feature computation
3. **Multi-model ensemble**: Combine multiple algorithms
4. **Causal inference**: Move beyond prediction to understand causation
5. **Prescriptive analytics**: Recommend optimal intervention strategies

## 📚 Additional Resources

- [Databricks MLflow Documentation](https://docs.databricks.com/mlflow/index.html)
- [Feature Engineering in Unity Catalog](https://docs.databricks.com/machine-learning/feature-store/index.html)
- [Model Serving Guide](https://docs.databricks.com/machine-learning/model-serving/index.html)
- [Lakehouse Monitoring](https://docs.databricks.com/lakehouse-monitoring/index.html)

## 💡 Key Takeaways

1. **MLflow + Unity Catalog** = Complete model lifecycle management
2. **Feature Store** ensures consistency between training and serving
3. **Champion/Challenger** pattern enables safe model updates
4. **Monitoring** is essential for production ML success
5. **Business impact** is the ultimate measure of success

---

**Questions or feedback?** This notebook was adapted from the [Databricks MLflow End-to-End Example](https://docs.databricks.com/aws/en/notebooks/source/mlflow/mlflow-classic-ml-e2e-mlflow-3.html) for healthcare payer use cases.